In [1]:
!pip install -q mtcnn onnx onnxruntime

In [2]:
import os
import cv2
import pickle
import numpy as np
import pandas as pd
from mtcnn import MTCNN
import onnxruntime as ort
import matplotlib.pyplot as plt
from skimage import transform as trans

Using TensorFlow backend.


In [3]:
def normalize(arr: np.array):
    x_norm = np.linalg.norm(arr, axis=1, keepdims=True)
    arr = arr/x_norm
    return arr

In [4]:
def load_data(folder_path):
    names = os.listdir(folder_path)
    if '.ipynb_checkpoints' in names:
        names.remove(".ipynb_checkpoints")
    data = {}
    total_face = 0
    for roots, dirs, files in os.walk(folder_path):
        if roots.find(".ipynb_checkpoints") == -1:
            name = roots.split('/')[-1]
            if name in names: 
                data[name] = []
                for file in files:
                    total_face += 1
                    data[name].append(os.path.join(roots, file))
    print('total face:', total_face)
    return data

In [5]:
def aligned_face(img, landmark):
    src = np.array([
        [30.2946, 51.6963],
        [65.5318, 51.5014],
        [48.0252, 71.7366],
        [33.5493, 92.3655],
        [62.7299, 92.2041] ], dtype=np.float32)
    dst = np.array(landmark, dtype=np.float32).reshape(5, 2)
    tform = trans.SimilarityTransform()
    tform.estimate(dst, src)
    M = tform.params[0:2,:]
    aligned = cv2.warpAffine(img, M, (112, 112), borderValue = 0)
    return aligned

In [6]:
def face_encodings(extractor, img):
    img = img.transpose((2, 0, 1))
    inputs = np.expand_dims(img, axis=0)
    input_name = extractor.get_inputs()[0].name
    embedding = extractor.run(None, {input_name: inputs})[0]
    outputs = normalize(embedding).flatten()
    return outputs

In [7]:
# load face data from database
folder_path = 'database'
data = load_data(folder_path)

total face: 28


In [8]:
# load MTCNN detector
detector = MTCNN()
# load mobilefacenet model
onnx_path = "mobilefacenet/mobilefacenet.onnx"
extractor = ort.InferenceSession(onnx_path)

In [9]:
allembeddings = np.zeros([len(data), 128])
for j, name in enumerate(data.keys()):
    embeddings = np.zeros([len(data[name]), 128])
    for i, file in enumerate(data[name]):
        print(file)
        imgBGR = cv2.imread(file, cv2.IMREAD_COLOR).astype('float32')
        imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)
        detection = detector.detect_faces(imgRGB)[0]
        landmark = np.array(list(detection['keypoints'].values()))
        aligned = aligned_face(imgRGB, landmark)
        embeddings[i] = face_encodings(extractor, aligned)
    allembeddings[j] = embeddings.mean(0)

database/TsaiIngWen/TsaiIngWen1.png
database/TsaiIngWen/TsaiIngWen4.jpg
database/TsaiIngWen/TsaiIngWen3.jpg
database/TsaiIngWen/TsaiIngWen2.jpg
database/WangShihChien/WangShihChien4.jpg
database/WangShihChien/WangShihChien1.jpg
database/WangShihChien/WangShihChien3.jpg
database/WangShihChien/WangShihChien2.jpg
database/MaYingJeou/MaYingJeou3.jpg
database/MaYingJeou/MaYingJeou4.jpg
database/MaYingJeou/MaYingJeou2.jpg
database/MaYingJeou/MaYingJeou1.jpg
database/ChenShihChung/ChenShihChung1.jpg
database/ChenShihChung/ChenShihChung3.jpg
database/ChenShihChung/ChenShihChung2.jpg
database/ChenShihChung/ChenShihChung4.jpg
database/Trump/Trump4.jpg
database/Trump/Trump1.jpg
database/Trump/Trump3.jpg
database/Trump/Trump2.jpg
database/Biden/Biden3.jpg
database/Biden/Biden4.jpg
database/Biden/Biden1.jpeg
database/Biden/Biden2.jpg
database/HanKuoYu/HanKuoYu1.jpg
database/HanKuoYu/HanKuoYu4.jpg
database/HanKuoYu/HanKuoYu3.jpg
database/HanKuoYu/HanKuoYu2.jpg


In [10]:
db = {'names': list(data.keys()), 'embeddings': allembeddings}
with open('db.pkl', 'wb') as file:
    pickle.dump(db, file)